# How to pass through arguments from one step to the next

# 1. Why Passing Data Matters in Chains

In LangChain, we often need to send data through different steps in a sequence, known as a chain. Sometimes, you might want certain data to continue unchanged through each step. For this, RunnablePassthrough comes into play. When combined with RunnableParallel, it becomes easy to run multiple processes at once while keeping data intact for certain steps.

# 2. Runnable Classes Overview
RunnablePassthrough: This class passes data through as-is. It’s ideal when you want to retain the original data in a specific key and use it later.

RunnableParallel: This class enables multiple tasks to execute in parallel, such as processing data in one step while passing other data through unchanged.

# 3. Example: Basic Usage of RunnablePassthrough and RunnableParallel
Here’s a simple example where we have two keys, passed and modified. passed uses RunnablePassthrough to keep the original input unchanged, while modified uses a function to increment a number:

In [2]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),   # Keeps original data unchanged
    modified=lambda x: x["num"] + 1 # Increments the number in "num"
)

result = runnable.invoke({"num": 1})
print(result) # Output: {'passed': {'num': 1}, 'modified': 2}


{'passed': {'num': 1}, 'modified': 2}


passed retains {'num': 1}, showing how RunnablePassthrough keeps data intact.

modified increments num by 1, resulting in modified: 2.

# 4. Real-World Example: Using RunnablePassthrough with a Prompt

In the second example, we set up a retrieval chain that searches a vector store for context. Here’s how it works:

Vector Store and Retriever: We use FAISS, a vector store, to store and retrieve context.

Prompt and Model: We create a template prompt that requires context and question keys and pass the prompt to a model to generate an answer.


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(["harrison worked at kensho"], embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

answer = retrieval_chain.invoke("where did harrison work?")
print(answer) # Output: "Harrison worked at Kensho."


Harrison worked at Kensho.


Explanation:

Retriever fetches relevant information from FAISS, providing context.

RunnablePassthrough passes the user’s question ("where did harrison work?") directly to the question key.

Prompt and Model use the context and question to generate an answer.